# Classificador géneres de música

Com a exemple d'utilització de models que treballen en audio anem a fer un classificador de gèneres de música. Per a això farem servir el dataset GTZAN, un dataset de 1000 mostres d'àudio etiquetades amb el gènere de la música.

## Instal·lació de llibreries
Per a poder executar aquest notebook necessitarem instal·lar les següents llibreries:

In [50]:
%pip install transformers datasets librosa soundfile torch accelerate evaluate

  Using cached evaluate-0.4.1-py3-none-any.whl (84 kB)
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Carreguem el dataset

In [ ]:
from datasets import load_dataset

gtzan = load_dataset("marsyas/gtzan", "all", trust_remote_code=True)
gtzan

Com podem veure, el dataset consta de 99 mostres d'àudio etiquetades amb el gènere de la música.

Els audios estan en format de 22050 Hz, per a poder-los processar amb el model necessitarem convertir-los a un format que pugui ser processat pel model (normalment 16kHz). Això ho farem amb la classe `Audio` de la llibreria datasets.

In [ ]:
from datasets import Audio

gtzan = gtzan.cast_column("audio", Audio(sampling_rate=16000))

## Creació del dataset de `test`

Per a poder avaluar el model necessitarem un dataset de test. Per a això dividirem el dataset en dos parts, una per a entrenar el model i una altra per a avaluar-lo.

In [ ]:
gtzan = gtzan["train"].train_test_split(seed=42, shuffle=True, test_size=0.1)
gtzan

Una vegada separat el dataset en dos parts, el dataset de test contindrà 100 mostres d'àudio.

A continuació mostrarem una mostra del dataset de test.

In [ ]:
gtzan['train'][0]

De cada mostra del dataset de test podem veure que tenim les següents dades:
- `audio`: El path a l'arxiu d'àudio.
- `array`: L'àudio en format d'array. El valor de cada element de l'array representa l'amplitud de l'ona en un instant de temps. Com el valor de samplig és de 16000 Hz, aquest array tindrà 16000 elements per segon.
- `genre`: El gènere de la música com a enter. Podem utilitzar el métode `int2str()` del `feature` _genre()_ per a obtenir el gènere en format llegible.

In [ ]:
int2str = gtzan["train"].features["genre"].int2str
int2str(gtzan['train'][0]['genre'])

## Testeig del model sense entrenar

Abans de començar a entrenar el model, testejarem el model sense entrenar per a veure com es comporta. Utilitzarem el model `distilhubert`, un model pre-entrenat per a classificar audio i lleuger de refinar.

Per a utilitzar el model farem servir la classe `pipeline` de la llibreria transformers.

In [42]:
from transformers import pipeline

classifier = pipeline(
    "audio-classification", model="ntu-spml/distilhubert"
)

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at ntu-spml/distilhubert and are newly initialized: ['classifier.bias', 'classifier.weight', 'encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [43]:
classifier(gtzan['train'][0]['audio'])

[{'score': 0.5038169026374817, 'label': 'LABEL_1'},
 {'score': 0.4961830973625183, 'label': 'LABEL_0'}]

Anem a calcular la precisió del model sense entrenar. Per a això farem servir el dataset de test.

El primer que farem serà calcular les prediccions del model per a cada mostra del dataset de test.

A continuació mostrarem les prediccions del model per a la primera mostra del dataset de test.

In [44]:
predictions = [classifier(sample['audio']) for sample in gtzan['test']]
predictions[0]

KeyboardInterrupt: 

Un cop tenim les prediccions del model, les compararem amb les etiquetes reals per a calcular la precisió del model.

A continuació mostrarem la precisió del model.

In [ ]:
from sklearn.metrics import accuracy_score

y_true = [f"LABEL_{sample['genre']}" for sample in gtzan['test']]
y_pred = [prediction[0]['label'] for prediction in predictions]

accuracy_score(y_true, y_pred)

## Preprocessament del dataset

Com podem veure, el model sense entrenar té una precisió del 10%, molt poc. Això és degut a que el model no ha estat entrenat amb el dataset GTZAN.

Per a entrenar el model farem servir la classe `Trainer` de la llibreria transformers. Aquesta classe ens permet entrenar models de manera senzilla i eficient.

Mentre que amb altres models necessitem un model i un `Tokenizer` en aquest cas farem servir un `model` i un feature_extractor`. Aquesta classe ens permetrà processar les mostres d'àudio per a convertir-les en un format que pugui ser processat pel model.

A continuació crearem el `feature_extractor` i que farem servir per a entrenar el model. 

In [ ]:
from transformers import AutoFeatureExtractor

model_id = "ntu-spml/distilhubert"
feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_id, do_normalize=True, return_attention_mask=True
)

A continuació processarrem les mostres d'àudio, convertint-les en un format que pugui ser processat pel model. En el nostre cas, retallarem les mostres d'àudio a 30 segons utilitzant les opcions `max_length` i `padding` del `feature_extractor` i llevarem les dades que no ens interessen del dataset amb el mètode `remove_columns`.

In [46]:
max_duration = 30.0


def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        truncation=True,
        return_attention_mask=True,
    )
    return inputs

In [47]:
gtzan_encoded = gtzan.map(
    preprocess_function,
    remove_columns=["audio", "file"],
    batched=True,
    batch_size=100,
    num_proc=1,
)
gtzan_encoded

Map:   0%|          | 0/899 [00:00<?, ? examples/s]

NameError: name 'feature_extractor' is not defined

Renomenarem la columna `genre` a `label` per a que el `Trainer` pugui identificar-la com a columna de labels.

In [ ]:
gtzan_encoded = gtzan_encoded.rename_column("genre", "label")

Per últim abans de començar a entrenar el model, crearem un diccionari en les correspondències entre els noms dels gèneres i els seus valors enters, per a que el `Trainer` pugui identificar-los i permetre un canvi ràpid entre els dos formats.

In [ ]:
id2label = {
    str(i): int2str(i)
    for i in range(len(gtzan_encoded["train"].features["label"].names))
}
label2id = {v: k for k, v in id2label.items()}

id2label["7"]

## Entrenament del model

A continuació crearem el model que anem a entrenar.

In [ ]:
from transformers import AutoModelForAudioClassification

model = AutoModelForAudioClassification.from_pretrained(
    model_id, num_labels=len(id2label)
)

A continuació crearem els `TrainerArguments`, que ens permetrà configurar el `Trainer` per a entrenar el model.

In [48]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    "test",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

ValueError: --load_best_model_at_end requires the save and eval strategy to match, but found
- Evaluation strategy: IntervalStrategy.EPOCH
- Save strategy: IntervalStrategy.STEPS

A continuació crearem el `Trainer`, classe que s'encarregarà de fer l'entrenament del model, i l'executarem.

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=gtzan_encoded["train"],
    eval_dataset=gtzan_encoded["test"],
    tokenizer=feature_extractor,
    compute_metrics=accuracy_score,
)
trainer.train()

## Testeig del model entrenat

Un cop entrenat el model, testejarem el model amb el dataset de test per a veure com es comporta.

A continuació mostrarem la precisió del model.

In [49]:
trainer.evaluate()

ValueError: Trainer: evaluation requires an eval_dataset.

## Classificació d'una cançó

Un cop creat el pipeline podem fer servir el mètode `classifier` per a classificar una cançó. Aquest mètode rep com a paràmetre el path a la cançó que volem classificar i retorna el gènere de la cançó.

Previament necessitarem música per a poder-la classificar. A continuació teniu alguns enllaços de música lliure per a poder-la descarregar i fer servir en aquest notebook:

- [Música clàssica](https://freemusicarchive.org/genre/Classical)
- [Música electrònica](https://freemusicarchive.org/genre/Electronic)
- [Música pop](https://freemusicarchive.org/genre/Pop)
- [Música rock](https://freemusicarchive.org/genre/Rock)
- [Música jazz](https://freemusicarchive.org/genre/Jazz)

També proporcionem enllaços directes que a alguns `samples` que podeu descarregar en la llibreria requests:


In [ ]:


musica = [
    "https://archive.org/download/cd_the-very-best-of-frank-sinatra_frank-sinatra-frank-nancy-sinatra/disc1/01.07.%20Frank%20Sinatra%20-%20Fly%20Me%20to%20the%20Moon%20%28In%20Other%20Words%29_sample.mp3",
    "https://archive.org/download/cd_random-access-memories_daft-punk/disc1/05.%20Daft%20Punk%20-%20Instant%20Crush_sample.mp3",
    "https://freemusicarchive.org/file/music/no_curator/Blue_Dot_Sessions/Organisms/Organisms_-_Blue_Dot_Sessions.mp3",
    "https://archive.org/download/cd_meteora_linkin-park/disc1/03.%20Linkin%20Park%20-%20Somewhere%20I%20Belong_sample.mp3"
    "https://ia800101.us.archive.org/21/items/cd_debbie-does-dallas_andrew-sherman/disc1/02.%20Andrew%20Sherman%20-%20Overture_sample.mp3",
    "https://archive.org/download/cd_greatest-hits_journey/Journey%20%281988%29%20-%20Journey%27s%20Greatest%20Hits%20%5BFLAC%5D/02%20-%20Don%27t%20Stop%20Believin%27_sample.mp3",
    "https://archive.org/download/cd_generator_bad-religion/disc1/01.%20Bad%20Religion%20-%20Generator_sample.mp3",
    "https://archive.org/download/geniesduclassique_vol1no12/1-09%20Concerto%20Brandeburghese%20No.3%20-%20Adagio.mp3"
]

In [ ]:
result = classifier(musica[0])